In [1]:

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("How are you?")

AIMessage(content="I'm just a computer program, so I don't have feelings or emotions. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 11, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-7f1e4ae9-a964-4996-81c8-3625c7f04707-0')

# Using Langsmith


In [3]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
llm.predict("How many people live in USA?", callbacks=[tracer])

/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'As of 2021, the estimated population of the United States is approximately 331 million people.'

In [4]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    llm.predict("How many people live in Germany?")

In [5]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["my-llm-tag"])
prompt = PromptTemplate.from_template("Say {input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])

chain("Hello, World!", tags=["shared-tags"])

/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'input': 'Hello, World!', 'text': 'Hello, World!'}

In [6]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("my_group_name") as group_manager:
    pass

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("my_group") as group_manager:
    chain.run(question="What is your name?", callbacks=group_manager)
    chain.run(question="What is your quest?", callbacks=group_manager)
    chain.run(question="What is your favorite color?", callbacks=group_manager)

/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [7]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x11b0586d0>

In [8]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs

<generator object Client.list_runs at 0x11b058660>

In [9]:
for run in todays_runs:
    print(run)

id=UUID('dc3c5b4b-b9c0-4d3b-ba21-ac9819a8c574') name='ChatOpenAI' start_time=datetime.datetime(2024, 4, 11, 11, 57, 25, 7306) run_type='llm' end_time=datetime.datetime(2024, 4, 11, 11, 57, 26, 22430) extra={'invocation_params': {'model': 'gpt-3.5-turbo', 'model_name': 'gpt-3.5-turbo', 'stream': False, 'n': 1, 'temperature': 0.9, '_type': 'openai-chat', 'stop': None}, 'options': {'stop': None}, 'batch_size': 1, 'metadata': {'system_fingerprint': 'fp_c2295e73ad'}} error=None serialized=None events=[{'name': 'start', 'time': '2024-04-11T11:57:25.007306+00:00'}, {'name': 'end', 'time': '2024-04-11T11:57:26.022430+00:00'}] inputs={'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the answer to What is your favorite color??'}}]} outputs={'llm_output': {'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad'}, 'run': None, 'generations': [{'text': 'My favorite color is blue.', 'generation_info':